In [3]:
import torch
torch.cuda.is_available()

/viscam/u/ksarge/mambaforge/envs/u2net/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [107]:
# !ls '/viscam/u/ksarge/bike_symbols/0105/0105_Updated Annotated Bike Symbols'
!ls '/viscam/u/ksarge/bike_symbols/1222/1222_bike_processed/San Jose CA'

'Street View 360_San Jose CA_1198_37.32451385_-121.88069857.jpg'
'Street View 360_San Jose CA_1198_37.32451385_-121.88069857.json'
'Street View 360_San Jose CA_1199_37.32454018_-121.88050550.jpg'
'Street View 360_San Jose CA_1199_37.32454018_-121.88050550.json'
'Street View 360_San Jose CA_1202_37.32469533_-121.88028048.jpg'
'Street View 360_San Jose CA_1202_37.32469533_-121.88028048.json'
'Street View 360_San Jose CA_1207_37.32446228_-121.88044697.jpg'
'Street View 360_San Jose CA_1207_37.32446228_-121.88044697.json'
'Street View 360_San Jose CA_1370_37.32785258_-121.87158620.jpg'
'Street View 360_San Jose CA_1370_37.32785258_-121.87158620.json'
'Street View 360_San Jose CA_1376_37.33458186_-121.89944204.jpg'
'Street View 360_San Jose CA_1376_37.33458186_-121.89944204.json'
'Street View 360_San Jose CA_1377_37.33468121_-121.89924753.jpg'
'Street View 360_San Jose CA_1377_37.33468121_-121.89924753.json'
'Street View 360_San Jose CA_1403_37.33361984_-121.87427349.jpg'
'Street View 360_S

In [162]:
import os
import glob

DATA_PATTERNS = [
    "/viscam/u/ksarge/bike_symbols/1222/*/*png",
    "/viscam/u/ksarge/bike_symbols/1222/*/*jpg",
    "/viscam/u/ksarge/bike_symbols/0105/*/*jpg",
    "/viscam/u/ksarge/bike_symbols/0105/*/*png",
    "/viscam/u/ksarge/bike_symbols/1022/*/*jpg",
    "/viscam/u/ksarge/bike_symbols/1022/*/*png",
    "/viscam/projects/streetview/bike_symbol_segmentation/data/*png",
    "/viscam/projects/streetview/bike_symbol_segmentation/data/*jpg"
]
LABEL_PATTERNS = [
    "/viscam/u/ksarge/bike_symbols/*/*/*json",
    "/viscam/u/ksarge/bike_symbols/*/*/*/*json",
    "/viscam/projects/streetview/bike_symbol_segmentation/data/*json"
]

def get_files(patterns):
    files = [
        file
        for pattern in patterns
        for file in glob.glob(pattern)]
    return files

In [163]:
data_files = get_files(DATA_PATTERNS)
len(data_files)

1171

In [164]:
import collections

data_files = get_files(DATA_PATTERNS)
data_files = [data_file for data_file in data_files if '_mask' not in data_file]

label_files = get_files(LABEL_PATTERNS)

# uid_to_file = collections.defaultdict(list)
# for data_file in data_files:
#     uid_to_file[os.path.basename(data_file)].append(data_file)
    
# for key in uid_to_file:
#     files = uid_to_file[key]
#     # if any('/viscam/projects/streetview/' in file for file in files):
#     if len(files) > 1:
#         print(files)
#         raise

In [165]:
data_files

['/viscam/u/ksarge/bike_symbols/1222/1222_Updated Annotated Bike Symbols/Street View 360_Minneapolis MN_1015_44.97287275_-93.27761629.jpg',
 '/viscam/u/ksarge/bike_symbols/1222/1222_Updated Annotated Bike Symbols/Street View 360_San Francisco CA_586_37.76950634_-122.41136669.jpg',
 '/viscam/u/ksarge/bike_symbols/1222/1222_Updated Annotated Bike Symbols/Street View 360_San Francisco CA_838_37.77223067_-122.41454775.jpg',
 '/viscam/u/ksarge/bike_symbols/1222/1222_Updated Annotated Bike Symbols/Street View 360_San Francisco CA_10_37.76342371_-122.42006333.jpg',
 '/viscam/u/ksarge/bike_symbols/1222/1222_Updated Annotated Bike Symbols/Street View 360_San Francisco CA_450_37.76646861_-122.39666960.jpg',
 '/viscam/u/ksarge/bike_symbols/1222/1222_Updated Annotated Bike Symbols/Street View 360_San Francisco CA_579_37.76923260_-122.41091125.jpg',
 '/viscam/u/ksarge/bike_symbols/1222/1222_Updated Annotated Bike Symbols/Street View 360_San Francisco CA_456_37.76653417_-122.39555260.jpg',
 '/viscam

In [178]:
def get_uid(path):
    uid = os.path.basename(os.path.splitext(path)[0])
    return uid

data_uids = {get_uid(data_file) for data_file in data_files}
label_uids = {get_uid(label_file) for label_file in label_files}

print(len(data_files), len(data_uids))
print(len(label_files), len(label_uids))
print(len(data_uids.intersection(label_uids)))

uid_to_data_file = {get_uid(data_file): data_file for data_file in data_files}
uid_to_label_file = {get_uid(label_file): label_file for label_file in label_files}

uids = set.intersection(set(uid_to_data_file), set(uid_to_label_file))
labels = []
for uid in uids:
    labels.append((uid_to_data_file[uid], uid_to_label_file[uid]))

772 541
771 539
538


In [181]:
import json
label_file = labels[0][1]

with open(label_file, 'r') as fp:
    label_json = json.load(fp)
    
label_json['imageData']

'...'